# Academy of Py Data Analysis - Miriam Berkowitz<br>
### Observed Trends:<br>
1)  Schools that spent more per student had lower overall passing rates than the schools that spent the least. The schools that spent the least per student had the highest overall passing rates.
<br>
2)  Larger schools had worse passing rates than smaller schools.
<br>
3)  Charter schools had better passing rates than district schools.


In [1]:
#Dependencies
import pandas as pd

In [2]:
#store filepath in variable
file1 = "../raw_data/schools_complete.csv"
file2 = "../raw_data/students_complete.csv"

#Read CSV files
schools_df = pd.read_csv(file1, encoding = "utf-8")
students_df = pd.read_csv(file2, encoding = "utf-8")

### District Summary

In [3]:
#compute district summary
total_schools = schools_df["School ID"].count()
total_students = students_df["Student ID"].count()
total_budget = schools_df["budget"].sum()
avg_math_score = round(students_df["math_score"].mean(),2)
avg_reading_score = round(students_df["reading_score"].mean(),2)

#MATH: assume the passing is a score of greater than 70
math_passed_table = students_df.loc[students_df["math_score"]> 70,:]

percent_passed_math = round((math_passed_table["Student ID"].count() / total_students),2) * 100
percent_passed_math

72.0

In [4]:
#READING: assume the passing is a score of greater than 70
reading_passed_table = students_df.loc[students_df["reading_score"]> 70,:]
percent_passed_reading = round((reading_passed_table["Student ID"].count() / total_students),2) * 100
percent_passed_reading

#compute overall passing rate -- average of math and reading passage averages
overall_passing_rate = (percent_passed_math + percent_passed_reading)/2
overall_passing_rate

83.0

In [6]:
#create a table with all the summary data computed above
summary_table = pd.DataFrame({"Total Schools": [total_schools], 
                              "Total Students": [total_students], 
                              "Total Budget": [total_budget], 
                              "Average Math Score": [avg_math_score], 
                              "Average Reading Score": [avg_reading_score], 
                              "Percentage Passing Math": [percent_passed_math], 
                              "Percentage Passing Reading": [percent_passed_reading], 
                              "Overall Passing Rate": [overall_passing_rate]})

#summary_table
#reorder the columns into the preferred order
summary_table = summary_table[["Total Schools",
                              "Total Students",
                              "Total Budget",
                              "Average Math Score",
                              "Average Reading Score",
                              "Percentage Passing Math",
                              "Percentage Passing Reading",
                              "Overall Passing Rate"]]


#format the budget as dollars and the passing rate fields as percents 
summary_table['Total Budget'] = summary_table['Total Budget'].map('${:,.0f}'.format)
summary_table['Percentage Passing Math'] = summary_table['Percentage Passing Math'].map('{:.0f}%'.format)
summary_table['Percentage Passing Reading'] = summary_table['Percentage Passing Reading'].map('{:.0f}%'.format)
summary_table['Overall Passing Rate'] = summary_table['Overall Passing Rate'].map('{:.0f}%'.format)

summary_table

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percentage Passing Math,Percentage Passing Reading,Overall Passing Rate
0,15,39170,"$24,649,428",78.99,81.88,72%,83%,78%


### School Summary

In [7]:
#rename the school name in both tables so that they are the same column name
schools_df = schools_df.rename(columns={"name":"school_name"})
students_df = students_df.rename(columns={"school":"school_name"})


In [8]:
#merge the school and students table, joining on school_name; inner join
merge_table = pd.merge(schools_df, students_df, on="school_name")

#compute average reading and math scores by school name
sch_avg_scores = pd.DataFrame(students_df.groupby("school_name").mean())

sch_avg_scores = sch_avg_scores.reset_index()
sch_avg_scores = sch_avg_scores.drop('Student ID', axis=1)

,School ID,school_name,type,size,budget,Student ID,name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [10]:
#make copy of schools data frame; then add the rest of the data for each school
school_summary_df = schools_df

#merge avg reading and math into school summary
school_summary_df = school_summary_df.merge(sch_avg_scores,on="school_name")

In [11]:
#compute "per student" budget; school budget/size
school_summary_df["Per Student Budget"] = round(school_summary_df["budget"] / school_summary_df["size"])
school_summary_df.head()

,School ID,school_name,type,size,budget,reading_score,math_score,Per Student Budget
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0


In [12]:
#compute percent passing math and reading and overall passing rate

#MATH: assume the passing is a score of over 70

#create table of just the students who passed math; then group by school
math_passed_table = students_df.loc[students_df["math_score"]> 70,:]
math_passed_table.head()
math_passed_totals = math_passed_table.groupby("school_name")['Student ID'].count()
   
math_passed_totals = math_passed_totals.reset_index()
#correct the column name to math_passed instead of Student ID
math_passed_totals = math_passed_totals.rename(columns={'Student ID':'math_passed'})
math_passed_totals.head()

,school_name,math_passed
0,Bailey High School,3216
1,Cabrera High School,1664
2,Figueroa High School,1880
3,Ford High School,1801
4,Griffin High School,1317


In [13]:
#create table of just the students who passed reading; then group by school
read_passed_table = students_df.loc[students_df["reading_score"]> 70,:]
read_passed_table.head()
read_passed_totals = read_passed_table.groupby("school_name")['Student ID'].count()
read_passed_totals = read_passed_totals.reset_index()
#correct the column name to read_passed instead of Student ID
read_passed_totals = read_passed_totals.rename(columns={'Student ID':'read_passed'})

read_passed_totals.head()

,school_name,read_passed
0,Bailey High School,3946
1,Cabrera High School,1744
2,Figueroa High School,2313
3,Ford High School,2123
4,Griffin High School,1371


In [14]:
#merge in math_passed_totals and read_passed_totals and then compute percent instead of total; then compute overall %
school_summary_df = school_summary_df.merge(math_passed_totals,on="school_name")
school_summary_df = school_summary_df.merge(read_passed_totals,on="school_name")
school_summary_df['math_passed'] = (school_summary_df['math_passed'] / school_summary_df["size"]) * 100 
school_summary_df['read_passed'] = (school_summary_df['read_passed'] / school_summary_df["size"]) * 100
school_summary_df['% Overall Passing'] = (school_summary_df['math_passed'] + school_summary_df['read_passed'])/2
school_summary_df.head()

,School ID,school_name,type,size,budget,reading_score,math_score,Per Student Budget,math_passed,read_passed,% Overall Passing
0,0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0,63.318478,78.813850,71.066164
1,1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0,63.750424,78.433367,71.091896
2,2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0,89.892107,92.617831,91.254969
3,3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0,64.746494,78.187702,71.467098
4,4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0,89.713896,93.392371,91.553134


In [15]:
#format and change column names
school_summary_df = school_summary_df.rename(columns= {"math_passed": "% Passing Math",
                                            "read_passed": "% Passing Reading",
                                                       "school_name": "School Name",
                                                       "type": "School Type",
                                                       "size": "Total Students",
                                                       "budget": "Total School Budget",
                                                       "reading_score": "Average Reading Score",
                                                       "math_score": "Average Math Score"})
                                     
school_summary_df['Per Student Budget'] = school_summary_df['Per Student Budget'].map('${:,.2f}'.format)
school_summary_df['Total School Budget'] = school_summary_df['Total School Budget'].map('${:,.2f}'.format)

# Drop the column with school ID 
school_summary_df.drop('School ID', axis=1,inplace=True)

In [16]:
#reorder the columns
school_summary_df = school_summary_df[["School Name",
                              "School Type",
                              "Total Students",
                              "Total School Budget",
                                "Per Student Budget",
                              "Average Math Score",
                              "Average Reading Score",
                              "% Passing Math",
                              "% Passing Reading",
                              "% Overall Passing"]]

#order alphabetically by school
school_summary_df = school_summary_df.sort_values("School Name")

#display the data
school_summary_df

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,71.631982
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867


### Top Performing Schools (By Passing Rate)

In [17]:
#sort the data to find the 5 highest performing schools
#sort on overall passing rate (descending) 
sorted_df = school_summary_df.sort_values(["% Overall Passing"], ascending=[False])

In [18]:
#top 5 (head) 
top_five = sorted_df.head()
top_five

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
5,Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,90.932983,93.254490,92.093736
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,91.683992,92.203742,91.943867
10,Wright High School,Charter,1800,"$1,049,400.00",$583.00,83.682222,83.955000,90.277778,93.444444,91.861111
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518
8,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,90.632319,92.740047,91.686183


### Bottom Performing Schools (by Passing Rate)


In [19]:
#note: the tail has the lowest performers, but they would be displayed reverse order, so resort ascending and display the first 5
sorted_df = school_summary_df.sort_values(["% Overall Passing"], ascending=[True])
sorted_df.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
11,Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,64.066017,77.744436,70.905226
0,Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,63.318478,78.813850,71.066164
12,Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,63.852132,78.281874,71.067003
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896
3,Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,64.746494,78.187702,71.467098


### Math Scores by Grade

In [20]:
#group the student data by school name and grade
grouped = students_df.groupby(['school_name','grade'])

In [21]:
#get the average math score from the grouped data
math_df = pd.DataFrame(grouped['math_score'].mean())
math_df = math_df.reset_index()

In [22]:
#reorganize the data for the output format
pivot_math = math_df.pivot(index='school_name',columns='grade',values='math_score')
pivot_math.head()

grade,10th,11th,12th,9th
school_name,,,,
Bailey High School,76.996772,77.515588,76.492218,77.083676
Cabrera High School,83.154506,82.765560,83.277487,83.094697
Figueroa High School,76.539974,76.884344,77.151369,76.403037
Ford High School,77.672316,76.918058,76.179963,77.361345
Griffin High School,84.229064,83.842105,83.356164,82.044010


In [23]:
pivot_math = pivot_math.reset_index()

pivot_math.columns.name = None

#reorder the grade columns so they are in order!
pivot_math = pivot_math[['school_name','9th','10th','11th','12th']]
pivot_math.set_index('school_name', inplace=True)
pivot_math

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


### Reading Scores by Grade

In [24]:
#get the average reading score from the grouped data
read_df = pd.DataFrame(grouped['reading_score'].mean())

In [25]:
read_df = read_df.reset_index()
#reorganize the data for the output format
pivot_read = read_df.pivot(index='school_name',columns='grade',values='reading_score')
pivot_read = pivot_read.reset_index()

#remove grade index column
pivot_read.columns.name = None
pivot_read.head()

,school_name,10th,11th,12th,9th
0,Bailey High School,80.907183,80.945643,80.912451,81.303155
1,Cabrera High School,84.253219,83.788382,84.287958,83.676136
2,Figueroa High School,81.408912,80.640339,81.384863,81.198598
3,Ford High School,81.262712,80.403642,80.662338,80.632653
4,Griffin High School,83.706897,84.288089,84.013699,83.369193


In [26]:
#reorder the grade columns so they are in order!
pivot_read = pivot_read[['school_name','9th','10th','11th','12th']]

pivot_read.set_index('school_name', inplace=True)
pivot_read

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


### Scores by School Spending

In [27]:
#put school summary data into schools_df
schools_df = school_summary_df

In [28]:
#set the per_student_budget to float in a new column
schools_df['Spending Range'] = schools_df['Per Student Budget'].replace('\$','',regex=True).astype('float')

#set 4 bins for budget per student
bins = [0,580,610,640,670]
group_names = ['Less than $580','\$580-$610','\$610-$640','More than $640']

#use group_indices to keep the data in the order I want; change to the group_names at the end
group_indices = ["1","2","3","4"]

schools_df["Spending Range Bins"] = pd.cut(schools_df["Spending Range"], bins, labels=group_indices)
schools_df.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Range,Spending Range Bins
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434,628.0,3
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518,582.0,2
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896,639.0,3
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,71.631982,644.0,4
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134,625.0,3


In [29]:
#group by the bins
school_groups = schools_df.groupby("Spending Range Bins")
school_means = school_groups.mean()

In [30]:
#remove unneeded columns
school_means.drop('Total Students', axis=1,inplace=True)
school_means.drop('Spending Range', axis=1,inplace=True)
school_means.reset_index(inplace=True)
school_means

,Spending Range Bins,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,1,83.274201,83.989488,90.932983,93.254490,92.093736
1,2,83.549353,83.903238,90.408972,92.974087,91.691529
2,3,79.474551,82.120471,74.474926,84.355203,79.415064
3,4,77.023555,80.957446,64.417757,78.198366,71.308062


In [31]:
#Replace the temporary index with the actual group names 
#this was done to keep the rows in the correct order
school_means['Spending Range Bins'] = school_means['Spending Range Bins'].replace(['1','2','3','4'], group_names)
school_means.set_index('Spending Range Bins', inplace=True)
school_means

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Range Bins,,,,,
Less than $580,83.274201,83.989488,90.932983,93.254490,92.093736
\$580-$610,83.549353,83.903238,90.408972,92.974087,91.691529
\$610-$640,79.474551,82.120471,74.474926,84.355203,79.415064
More than $640,77.023555,80.957446,64.417757,78.198366,71.308062


### Scores by School Size

In [32]:
#set 3 bins for school size
bins = [0,1500,2500,5000]
group_names = ['Small (<1500)','Medium (1500-2500)','Large (>2500)']
group_indices = ["1","2","3"]
schools_df["Size Bins"] = pd.cut(schools_df["Total Students"], bins, labels=group_indices)
schools_df.head()

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Range,Spending Range Bins,Size Bins
7,Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,64.630225,79.300643,71.965434,628.0,3,3
6,Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,89.558665,93.864370,91.711518,582.0,2,2
1,Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,63.750424,78.433367,71.091896,639.0,3,3
13,Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,65.753925,77.510040,71.631982,644.0,4,3
4,Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,89.713896,93.392371,91.553134,625.0,3,1


In [34]:
#group by the bins
school_groups = schools_df.groupby("Size Bins")
school_means = school_groups.mean()

#remove unneeded columns
school_means.drop('Total Students', axis=1,inplace=True)

school_means.reset_index(inplace=True)

In [35]:
#Replace the temporary index with the actual group names 
#this was done to keep the rows in the correct order
school_means['Size Bins'] = school_means['Size Bins'].replace(['1','2','3'], group_names)

school_means.set_index('Size Bins', inplace=True)
school_means

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Range
Size Bins,,,,,,
Small (<1500),83.664898,83.892148,90.676736,92.778720,91.727728,605.000000
Medium (1500-2500),83.359224,83.898984,90.175120,93.217267,91.696193,596.200000
Large (>2500),76.956733,80.966636,64.302528,78.324559,71.313543,643.571429


### Scores by School Type

In [42]:
#group by type
school_groups = schools_df.groupby("School Type")
school_means = school_groups.mean()

#remove unneeded columns
school_means.drop('Total Students', axis=1,inplace=True)
del school_means['Spending Range']
school_means

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
